In [1]:
cd ..

c:\SMIT3.0


เพิ่มการดึง Query ข้อมูลจาก Table AUDITResult, Classification_Finding 

In [17]:
from deep_translator import GoogleTranslator               
from Function import Cleansing_Input                       
import pandas as pd                                        
import pyodbc                                              
import re                                                  
from sentence_transformers import SentenceTransformer, util

modelPath = "./Model/SentenceTransformer"

model = SentenceTransformer(modelPath)

# Real Database Connection
# connect_db = pyodbc.connect(Driver = "ODBC Driver 17 for SQL Server",
#                             Server = "TONY",
#                             Database = "SMIT3",
#                             uid = 'Local_SMIT3.0',
#                             pwd = 'Tony123456',
#                             Trusted_Connection = 'yes')  

# Local Current Database
# connect_db = pyodbc.connect(Driver = "ODBC Driver 17 for SQL Server",
#                             Server = "TONY",
#                             Database = "SMIT3",
#                             uid = 'Local_SMIT3.0',
#                             pwd = 'Tony123456',
#                             Trusted_Connection = 'yes')  

# Local Used all Data
connect_db = pyodbc.connect(Driver = "ODBC Driver 17 for SQL Server",
                            Server = "TONY",
                            Database = "SMIT3-Real",
                            uid = 'Local_SMIT3.0',
                            pwd = 'Tony123456',
                            Trusted_Connection = 'yes')  
cursor = connect_db.cursor()

Cleansing_FindingDetails = pd.read_sql("SELECT * FROM [Cleansing_FindingDetails];", connect_db)

FindingNo = Cleansing_FindingDetails['FindingNo'].tolist()
FindingNo = ["-" if pd.isnull(x) else x for x in FindingNo]

Area = Cleansing_FindingDetails['Area'].tolist()
Area = ["-" if pd.isnull(x) else x for x in Area]

SubArea = Cleansing_FindingDetails['SubArea'].tolist()
SubArea = ["-" if pd.isnull(x) else x for x in SubArea]

Contractor = Cleansing_FindingDetails['Contractor'].tolist()
Contractor = ["-" if pd.isnull(x) else x for x in Contractor]

Tof = Cleansing_FindingDetails['TypeOfFinding'].tolist()
Tof = ["-" if pd.isnull(x) else x for x in Tof]

Topic = Cleansing_FindingDetails['Topic'].tolist()
Topic = ["-" if pd.isnull(x) else x for x in Topic]

Details = Cleansing_FindingDetails['Details'].tolist()
Details = ["-" if pd.isnull(x) else x for x in Details]

CleansingDetails = Cleansing_FindingDetails['CleansingDetails'].tolist()
CleansingDetails = ["-" if pd.isnull(x) else x for x in CleansingDetails]

TranslateDetails = Cleansing_FindingDetails['TranslateDetails'].tolist()
TranslateDetails = ["-" if pd.isnull(x) else x for x in TranslateDetails]

Insert Data From Excel to Database: FindingDetails

In [22]:
connect_db = pyodbc.connect(Driver = "ODBC Driver 17 for SQL Server",
                            Server = "TONY",
                            Database = "SMIT3",
                            uid = 'Local_SMIT3.0',
                            pwd = 'Tony123456',
                            Trusted_Connection = 'yes')  

cursor = connect_db.cursor()

Query = "DELETE FROM FindingDetails;"

cursor.execute(Query)

TotalData = (pd.read_csv('./SMIT_Data/TotalData.csv', encoding='utf-8'))

TotalOldFindingDetails = TotalData['Old'].tolist()[0]

FindingDetails = (pd.read_csv('./SMIT_Data/Raw_Safety_Audit.csv', encoding='utf-8'))[TotalOldFindingDetails:]

FindingNo = FindingDetails['No.'].tolist()
FindingNo = ["-" if pd.isnull(x) else x for x in FindingNo]

Company = FindingDetails['Company'].tolist()
Company = ["-" if pd.isnull(x) else x for x in Company]

Area = FindingDetails['Area'].tolist()
Area = ["-" if pd.isnull(x) else x for x in Area]

SubArea = FindingDetails['Sub Area'].tolist()
SubArea = ["-" if pd.isnull(x) else x for x in SubArea]

Reportor = FindingDetails['Reportor'].tolist()
Reportor = ["-" if pd.isnull(x) else x for x in Reportor]

Contractor = FindingDetails['Contractor'].tolist()
Contractor = ["-" if pd.isnull(x) else x for x in Contractor]

Tof = FindingDetails['Type of finding'].tolist()
Tof = ["-" if pd.isnull(x) else x for x in Tof]

Topic = FindingDetails['Topic'].tolist()
Topic = ["-" if pd.isnull(x) else x for x in Topic]

Details = FindingDetails['Details'].tolist()
Details = ["-" if pd.isnull(x) else x for x in Details]

CorrectiveAction = FindingDetails['Corrective Action'].tolist()
CorrectiveAction = ["-" if pd.isnull(x) else x for x in CorrectiveAction]

IssueDate = FindingDetails['Issue Date'].tolist()
IssueDate = ["-" if pd.isnull(x) else x for x in IssueDate]

DueDate = FindingDetails['Due Date'].tolist()
DueDate = ["-" if pd.isnull(x) else x for x in DueDate]

Status = FindingDetails['Status'].tolist()
Status = ["-" if pd.isnull(x) else x for x in Status]

FindingData = list(zip(FindingNo, Company, Area, SubArea, 
                      Reportor, Contractor, Tof, 
                      Topic, Details, CorrectiveAction,
                      IssueDate, DueDate, Status))

if len(FindingData) > 0:
    Query = "INSERT INTO [FindingDetails] VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"

    cursor.executemany(Query, FindingData)

    connect_db.commit()

In [20]:
import csv

Old_Size = len(FindingNo)

Head = ['Source', 'Old', 'Latest']

UpdateSize = ['FindingDetails', TotalOldFindingDetails, TotalOldFindingDetails+Old_Size]

with open('./SMIT_Data/TotalData.csv', 'w', newline='', encoding="utf-8") as f:
  write = csv.writer(f)
  write.writerow(Head)
  write.writerow(UpdateSize)

-------------------------------------------------------------------------------------

In [9]:
Prepared_FindingNo = []
Prepared_Area = []
Prepared_SubArea = []
Prepared_Contractor = []
Prepared_Tof = []
Prepared_Topic = []
Prepared_Details = []
Prepared_Frequency = []
Prepared_CleansingDetails = []
Prepared_Translate_Details = []
Prepared_ListFindingNo = []
MostMatch = []

Encode_Translate_Cleansing_Safety_Audit = model.encode(TranslateDetails)

Cosine_Sim = util.cos_sim(Encode_Translate_Cleansing_Safety_Audit, Encode_Translate_Cleansing_Safety_Audit)

# Index_Frequency = []
Size = len(Encode_Translate_Cleansing_Safety_Audit)

Sum = 0
for i in range(0, Size):
  Index_Most_Frequency = 0
  Count_Frequency = 0
  Max_Cosine = 0
  Index_Frequency = []
  if i not in MostMatch:
    # Index_Frequency.append(i)
    if (Tof[i] == "Unsafe Condition"):
      for j in range(0, Size):
        if i != j:
          if Cosine_Sim[i][j] > 0.6 and Tof[j] == "Unsafe Condition":
              Count_Frequency += 1 
              Index_Frequency.append(j) 
              if Cosine_Sim[i][j] > Max_Cosine:
                Max_Cosine = Cosine_Sim[i][j]
                Index_Most_Frequency = j
        
    elif (Tof[i] == "Unsafe Action"):
      for j in range(0, Size):
        if i != j:
          if Cosine_Sim[i][j] > 0.6 and Tof[j] == "Unsafe Action": 
            Count_Frequency += 1 
            Index_Frequency.append(j) 
            if Cosine_Sim[i][j] > Max_Cosine:
              Max_Cosine = Cosine_Sim[i][j]
              Index_Most_Frequency = j

    elif (Tof[i] == "HNM"):
      for j in range(0, Size):
        if i != j:
          if Cosine_Sim[i][j] > 0.7 and Tof[j] == "HNM": 
            Count_Frequency += 1 
            Index_Frequency.append(j) 
            if Cosine_Sim[i][j] > Max_Cosine: 
              Max_Cosine = Cosine_Sim[i][j]
              Index_Most_Frequency = j

    elif (Tof[i] == "Near Miss"):
      for j in range(0, Size):
        if i != j:
          if Cosine_Sim[i][j] > 0.7 and Tof[j] == "Near Miss": 
            Count_Frequency += 1 
            Index_Frequency.append(j) 
            if Cosine_Sim[i][j] > Max_Cosine:
              Max_Cosine = Cosine_Sim[i][j]
              Index_Most_Frequency = j

    elif (Tof[i] == "Accident"):
      for j in range(0, Size):
        if i != j:
          if Cosine_Sim[i][j] > 0.7 and Tof[j] == "Accident": 
            Count_Frequency += 1 
            Index_Frequency.append(j) 
            if Cosine_Sim[i][j] > Max_Cosine:   
              Max_Cosine = Cosine_Sim[i][j]
              Index_Most_Frequency = j
              
    if Count_Frequency == 0:
      Index_Frequency.append(i)
      Count_Frequency = 1 
      Index_Most_Frequency = i  

    if (Index_Most_Frequency not in MostMatch) and Details[Index_Most_Frequency] not in Prepared_Details:
      Prepared_FindingNo.append(int(FindingNo[Index_Most_Frequency]))
      Prepared_Area.append(Area[Index_Most_Frequency])
      Prepared_SubArea.append(SubArea[Index_Most_Frequency])
      Prepared_Contractor.append(Contractor[Index_Most_Frequency])
      Prepared_Tof.append(Tof[Index_Most_Frequency])
      Prepared_Topic.append(Topic[Index_Most_Frequency])
      Prepared_Details.append(Details[Index_Most_Frequency])
      Prepared_Frequency.append(Count_Frequency)
      Prepared_CleansingDetails.append(CleansingDetails[Index_Most_Frequency])
      Prepared_Translate_Details.append(TranslateDetails[Index_Most_Frequency])
      Prepared_ListFindingNo.append(Index_Frequency)
      MostMatch.append(Index_Most_Frequency)

    if Index_Most_Frequency in MostMatch and (Details[Index_Most_Frequency] in Prepared_Details and Area[Index_Most_Frequency] in Prepared_Area):
      Index = MostMatch.index(Index_Most_Frequency)
      Temp_Prepared_ListFindingNo = Prepared_ListFindingNo[Index]+Index_Frequency
      Temp_Prepared_ListFindingNo = list(set(Temp_Prepared_ListFindingNo))
      Prepared_ListFindingNo[Index] = Temp_Prepared_ListFindingNo
      Prepared_Frequency[Index] = len(Temp_Prepared_ListFindingNo)

# print(MostMatch)

In [11]:
from itertools import chain

result = len(set(list(chain.from_iterable(Prepared_ListFindingNo))))

print(result)

List = [832, 1028, 1348, 712, 936, 1161, 2028, 593, 915, 916, 1523, 633, 474, 636, 861]

2423


In [12]:
import csv

connect_db = pyodbc.connect(Driver = "ODBC Driver 17 for SQL Server",
                            Server = "TONY",
                            Database = "SMIT3",
                            uid = 'Local_SMIT3.0',
                            pwd = 'Tony123456',
                            Trusted_Connection = 'yes')      

print(len(Prepared_FindingNo))

ResponsePreparedFindingDetail = pd.read_sql("SELECT * FROM Prepared_FindingDetails;", connect_db)

ResponsePreparedFindingDetail = list(ResponsePreparedFindingDetail.itertuples(index=False, name=None))

print(len(ResponsePreparedFindingDetail))

if(len(ResponsePreparedFindingDetail) > 0):

    cursor = connect_db.cursor()
    Query = "DELETE FROM [Prepared_FindingDetails];"
    cursor.execute(Query)
    ResponsePreparedFindingDetail = []

Prepared_Safety_Audit = list(zip(Prepared_FindingNo, Prepared_Area, Prepared_SubArea, 
                            Prepared_Contractor, Prepared_Tof, Prepared_Topic, Prepared_Details, 
                            Prepared_Frequency, Prepared_CleansingDetails, Prepared_Translate_Details))

cursor = connect_db.cursor()
Query = "INSERT INTO [Prepared_FindingDetails] VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);"

cursor.executemany(Query, Prepared_Safety_Audit)
connect_db.commit()

Head = ['FindingNo', 'Area', 'SubArea', 'Contractor', 'TypeOfFinding', 'Topic', 'Details', 'Frequency', 'CleansingDetails', 'TranslateDetails']

with open('./SMIT_Data/Prepared_Safety_Audit.csv', 'w', newline='', encoding="utf-8") as f:
  write = csv.writer(f)
  write.writerow(Head)
  write.writerows(Prepared_Safety_Audit)

1600
1607
